In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dermatology-dataset-classification/dermatology_database_1.csv


# Assignment Details
You are required to explore and work with two different datasets to apply Logistic Regression and Multinomial Logistic Regression. The objective is to understand how Logistic Regression works for binary classification problems and how Multinomial Logistic Regression is used for multiclass (more than two classes) classification. Additionally, evaluate the models' performance using appropriate metrics.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Loading Dataset

In [3]:
df = pd.read_csv("/kaggle/input/dermatology-dataset-classification/dermatology_database_1.csv")

In [4]:
df.head()

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_and_elbow_involvement,scalp_involvement,...,disappearance_granular_layer,vacuolisation_damage_basal_layer,spongiosis,saw_tooth_appearance_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_mononuclear_infiltrate,band_like_infiltrate,age,class
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3


# Data Preprocessing 

In [5]:
df.shape

(366, 35)

In [6]:
df.dtypes

erythema                                int64
scaling                                 int64
definite_borders                        int64
itching                                 int64
koebner_phenomenon                      int64
polygonal_papules                       int64
follicular_papules                      int64
oral_mucosal_involvement                int64
knee_and_elbow_involvement              int64
scalp_involvement                       int64
family_history                          int64
melanin_incontinence                    int64
eosinophils_infiltrate                  int64
PNL_infiltrate                          int64
fibrosis_papillary_dermis               int64
exocytosis                              int64
acanthosis                              int64
hyperkeratosis                          int64
parakeratosis                           int64
clubbing_rete_ridges                    int64
elongation_rete_ridges                  int64
thinning_suprapapillary_epidermis 

In [7]:
df["age"].dtype

dtype('O')

In [8]:
df.isnull().sum()

erythema                               0
scaling                                0
definite_borders                       0
itching                                0
koebner_phenomenon                     0
polygonal_papules                      0
follicular_papules                     0
oral_mucosal_involvement               0
knee_and_elbow_involvement             0
scalp_involvement                      0
family_history                         0
melanin_incontinence                   0
eosinophils_infiltrate                 0
PNL_infiltrate                         0
fibrosis_papillary_dermis              0
exocytosis                             0
acanthosis                             0
hyperkeratosis                         0
parakeratosis                          0
clubbing_rete_ridges                   0
elongation_rete_ridges                 0
thinning_suprapapillary_epidermis      0
spongiform_pustule                     0
munro_microabcess                      0
focal_hypergranu

In [9]:
import category_encoders as ce

encoder = ce.OrdinalEncoder()
df["age"] = encoder.fit_transform(df[["age"]])


In [11]:
df["age"].dtype

dtype('int64')

In [12]:
df.columns

Index(['erythema', 'scaling', 'definite_borders', 'itching',
       'koebner_phenomenon', 'polygonal_papules', 'follicular_papules',
       'oral_mucosal_involvement', 'knee_and_elbow_involvement',
       'scalp_involvement', 'family_history', 'melanin_incontinence',
       'eosinophils_infiltrate', 'PNL_infiltrate', 'fibrosis_papillary_dermis',
       'exocytosis', 'acanthosis', 'hyperkeratosis', 'parakeratosis',
       'clubbing_rete_ridges', 'elongation_rete_ridges',
       'thinning_suprapapillary_epidermis', 'spongiform_pustule',
       'munro_microabcess', 'focal_hypergranulosis',
       'disappearance_granular_layer', 'vacuolisation_damage_basal_layer',
       'spongiosis', 'saw_tooth_appearance_retes', 'follicular_horn_plug',
       'perifollicular_parakeratosis', 'inflammatory_mononuclear_infiltrate',
       'band_like_infiltrate', 'age', 'class'],
      dtype='object')

In [13]:
corr_val = df.corrwith(df['class'])
corr_val.sort_values(ascending=False)

class                                  1.000000
fibrosis_papillary_dermis              0.526976
follicular_papules                     0.477813
perifollicular_parakeratosis           0.457025
follicular_horn_plug                   0.431972
exocytosis                             0.283433
spongiosis                             0.213401
focal_hypergranulosis                  0.063241
saw_tooth_appearance_retes             0.060444
polygonal_papules                      0.057785
melanin_incontinence                   0.057394
band_like_infiltrate                   0.057266
oral_mucosal_involvement               0.055805
vacuolisation_damage_basal_layer       0.055402
itching                                0.054772
age                                    0.011766
inflammatory_mononuclear_infiltrate   -0.020402
hyperkeratosis                        -0.054428
eosinophils_infiltrate                -0.062323
acanthosis                            -0.079771
koebner_phenomenon                    -0

In [14]:
# Sort correlation values and save the result
corr_val = corr_val.sort_values(ascending=False)

corr_threshold = 0.5

# Select columns with correlation >= 0.5
selected_cols = corr_val[abs(corr_val) >= 0.5].index.tolist()

print("Highly correlated columns (>= 0.5):")
print(selected_cols)


Highly correlated columns (>= 0.5):
['class', 'fibrosis_papillary_dermis', 'munro_microabcess', 'scalp_involvement', 'PNL_infiltrate', 'clubbing_rete_ridges', 'thinning_suprapapillary_epidermis']


# Spliting Data

In [21]:
x = df[[col for col in selected_cols if col != 'class']]
y = df['class']

In [23]:
x.shape, y.shape

((366, 6), (366,))

In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [30]:
x

array([[-0.39445573, -0.47897227, -0.57399979, -0.67103516, -0.62909261,
        -0.61332752],
       [-0.39445573,  0.8391018 ,  0.53170507,  0.55695918,  1.2659518 ,
         1.32182654],
       [-0.39445573, -0.47897227, -0.57399979, -0.67103516, -0.62909261,
        -0.61332752],
       ...,
       [-0.39445573, -0.47897227, -0.57399979, -0.67103516, -0.62909261,
        -0.61332752],
       [-0.39445573, -0.47897227, -0.57399979, -0.67103516, -0.62909261,
        -0.61332752],
       [-0.39445573,  2.15717586,  2.74311479,  0.55695918,  2.21347401,
         2.28940357]])

In [31]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [34]:
from sklearn.linear_model import LogisticRegression 

mlr = LogisticRegression(max_iter = 1000, multi_class="multinomial")
mlr.fit(x_train, y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [45]:
from sklearn.metrics import classification_report
train_predict = mlr.predict(x_train)
print("Training Data Results")
print(classification_report(y_train, train_predict, zero_division=0))
print(confusion_matrix(y_train, train_predict))


Training Data Results
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        81
           2       0.84      0.71      0.77        52
           3       0.49      0.95      0.64        59
           4       0.00      0.00      0.00        41
           5       0.98      1.00      0.99        42
           6       0.67      0.35      0.46        17

    accuracy                           0.76       292
   macro avg       0.66      0.67      0.64       292
weighted avg       0.70      0.76      0.71       292

[[81  0  0  0  0  0]
 [ 0 37 14  0  0  1]
 [ 0  0 56  0  1  2]
 [ 0  6 35  0  0  0]
 [ 0  0  0  0 42  0]
 [ 0  1 10  0  0  6]]


In [46]:
from sklearn.metrics import classification_report, confusion_matrix 
y_pred = mlr.predict(x_test)
print("Testing Data Results")
print(classification_report(y_test, y_pred, zero_division=0))
print(confusion_matrix(y_test, y_pred))


Testing Data Results
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        31
           2       1.00      0.78      0.88         9
           3       0.50      1.00      0.67        13
           4       0.00      0.00      0.00         8
           5       1.00      1.00      1.00        10
           6       0.00      0.00      0.00         3

    accuracy                           0.82        74
   macro avg       0.58      0.63      0.59        74
weighted avg       0.76      0.82      0.78        74

[[31  0  0  0  0  0]
 [ 0  7  2  0  0  0]
 [ 0  0 13  0  0  0]
 [ 0  0  8  0  0  0]
 [ 0  0  0  0 10  0]
 [ 0  0  3  0  0  0]]
